# Qopen: Separation of intrinsic attenuation and scattering, site and source terms

## Qopen practical using the 2019 Ridecrest Earthquake Sequence dataset

*Author: Tom Eulenfeld*

Qopen can be used to estimate the scattering and intrinsic contributions to the attenuation for a local to regional dataset.  In addition, Qopen can be used to estimate relative site gains as a function of frequency and to estimate source spectra and source parameters, i.e., seismic moment, moment magnitude, corner frequency, and stress drop, for small to moderate earthquakes.

This notebook was created for the 13th Munich Earth Skience School (2023).

### Dataset

The dataset used in this notebook and included in the repository is available at https://scedc.caltech.edu/data/stressdrop-ridgecrest.html
and was used as a benchmark in the "Community Stress Drop Validation Study using the 2019 Ridgecrest Earthquake Sequence".

Dataset References:
 * Baltay, A., Abercrombie, R. E., Taira, T. (2021), A Community Stress Drop Validation Study Using the 2019 Ridgecrest Earthquake Dataset, SSA Annual Meeting 2021.
 * Trugman, D. T. (2020), Stress‐Drop and Source Scaling of the 2019 Ridgecrest, California, Earthquake Sequence. Bulletin of the Seismological Society of America 2020; 110 (4): 1859–1871. doi: 10.1785/0120200009
 * Baltay, A. S., R. E. Abercrombie, T. Taira, A community stress drop validation study using the 2019 Ridgecrest Earthquake Sequence, Seismological Research Letters (in preparation)

The dataset consists of 56 earthquakes from the 2019 earthquake sequence. For most of this practical, we will use 8 selected events from these 56 events. Out of the 34 stations provided in the inventory, we will only use 6 stations to limit the runtime of Qopen

### Setup

To work through the practical, please download or clone this repository.
You will need the following Python packages: `jupyter obspy qopen cartopy`.
For ObsPy versions `<1.5` you will additionally need the package `obspycsv`.
Finally, start the Jupyter notebook server with `jupyter notebook`.
One of many possibilities for installation is via conda:

```
conda create -c conda-forge -n practical jupyter obspy cartopy statsmodels
conda activate practical
pip install qopen
#pip install obspycsv  # for ObsPy<1.5
jupyter notebook
```

### Topics

The notebook covers the following topics:

1. Plot event catalog with ObsPy and in a custom plot including depth sections
2. Plot waveforms and calculate the seismic envelope and spectral energy density
3. Understand impact of intrinsic attenuation and scattering on the seismic envelope
4. Use Qopen to estimate intrinsic attenuation and scattering strength for a local dataset, load and plot results
5. Calculate coda Q for one event-station pair and compare it with the Q estimates for scattering and intrinsic attenuation
5. Estimate basic source parameters including the moment magnitude

Answers and code updates can be directly inserted into this notebook.

The slides of an introductory talk presented at the 2023 Skience workshop are located in this repository.
Additional resources are listed in the [Qopen repository](https://github.com/trichter/qopen).
Get help in the [Qopen category of the ObsPy forum](https://discourse.obspy.org/c/obspy-related-projects/qopen/15).
Bugs can be reported in the issue tracker of this repository.

## 1) Plot event catalog
First, let us get an impression of the dataset with ObsPy's on-board tools. Light colored triangles show all stations of the provided inventory, solid colored triangles show the 6 selected stations. Circles represent earthquake hypocenters, color codes the depth, size the magnitude.

In [ ]:
from obspy import read_events, read_inventory
from os.path import normpath

inv = read_inventory(normpath('data/stations.txt'))
inv6 = read_inventory(normpath('data/stations_selected.txt'))
events8 = read_events(normpath('data/events8.csz'))
events = read_events(normpath('data/events56.csz'))

print('6 selected stations in', inv6)
print()
print('Selected', events)

fig = inv.plot('local',  color='#f0b293', label=False, show=False)
inv6.plot('local', label=False, fig=fig, show=False)
events.plot('local', label=None, fig=fig);

In the following cell, we quickly plot a custom map with west-east and north-south depth slices. The `_events2array` function in the `obspy.io.csv` module is used to convert the ObsPy catalog object, modeled after the QUAKEML format, into a flat NumPy array.

Additionally the magnitude-time distribution of the catalog is plotted. All 56 events are plotted with transparency enabled, the selected 8 events are plotted in solid blue with black marker edges.

In [ ]:
import matplotlib.pyplot as plt
try:
    from obspy.io.csv import _events2array
except ImportError:
    from obspycsv import _events2array  # ObsPy<1.5


e = _events2array(events)
e8 = _events2array(events8)
print('NumPy array with event parameters', e8.dtype.names)
print(e8)

# size of circles scales with magnitude
print('\nMap and depth sections, size scales with magnitude')
kw1 = dict(s=4*e['mag']**2, color='C0', alpha=0.5)
kw2 = dict(s=4*e8['mag']**2, color='C0', edgecolors='k')
plt.subplot(221)
plt.scatter(e['lon'], e['lat'], **kw1)
plt.scatter(e8['lon'], e8['lat'], **kw2)
plt.ylabel('latitude')
plt.subplot(222)
plt.scatter(e['dep'], e['lat'], **kw1)
plt.scatter(e8['dep'], e8['lat'], **kw2)
plt.xlabel('depth (km)')
plt.subplot(223)
plt.scatter(e['lon'], e['dep'], **kw1)
plt.scatter(e8['lon'], e8['dep'], **kw2)
plt.gca().invert_yaxis()
plt.xlabel('longitude')
plt.ylabel('depth (km)')
plt.show()

print('Magnitude vs time, size scales with magnitude')
fig = plt.figure(figsize=(8, 2))
plt.scatter(e['time'], e['mag'], **kw1)
plt.scatter(e8['time'], e8['mag'], **kw2)
plt.ylabel('magnitude')
fig.autofmt_xdate()

## 2) Waveforms, envelopes, Green's functions

Let us look at some waveform data. We select an event and load and plot data from a specific station.

In [ ]:
from obspy import read

evid = '38445975'
sta = 'CLC'
print('IDs of 8 events are', ', '.join(e8['id']))
print(f'Select event {evid}')
ev = e[e['id'] == evid][0]
stream = read(normpath(f'data/waveforms/{evid}/*{sta}*.ms'))
stream.remove_sensitivity(inv)
stream.plot(size=(800, 300));

### Envelope

We are interested in the energy levels in the direct wave and the coda. We do not care about the phase, which cannot be modeled using radiative transfer. Therefore, as a first step, we convert a single component of the seismogram $s$ to its envelope using the Hilbert transform implemented in the SciPy package. The analytic signal $\tilde s$ can be calculated with the formula $$\tilde s = s + i \mathcal H (s)\,.\tag 1$$

The envelope (instantaneous amplitude) is simply the absolute value of the analytic (complex) signal. Equation (1) is implemented in the function `scipy.signal.hilbert`. The time array t is given in seconds relative to the origin time.

*Please adjust the x limit of the plot to a time window around the onset (e.g. 0s to 15s relative to origin time).*

In [ ]:
import numpy as np
from obspy import UTCDateTime as UTC
import scipy


def envelope(tr):
    analytic_signal = scipy.signal.hilbert(tr.data)  
    tr.data = np.abs(analytic_signal)
    return tr

tr = stream[0]
print(tr)

t = tr.times(reftime=UTC(str(ev['time'])))
plt.plot(t, tr.data, label='data')
env = envelope(tr.copy())
plt.plot(t, env.data, label='envelope')
plt.xlabel('time after rupture (s)')
plt.ylabel('ground velocity (m/s)')
plt.legend();

### Spectral energy density

To compare the envelope to the radiative transfer Green's function, it must be scaled to the spectral energy density. Before applying the Hilbert transform the signal is filtered with a bandpass-filter. The envelope above has units of m/s, the spectral energy density has units of Jm$^{-3}$Hz$^{-1}$. The narrowband envelopes $|\tilde s|$ of the three components must be squared, summed, and scaled to obtain the spectral energy density  $E$:
$$E = \frac{\rho \sum_{i=1}^3 |\tilde s_i|^2}{2C_\text E \Delta f} \tag 2$$
$\rho$ is the mean density. $\Delta f$ is the filter width, which can be approximated by the difference of the filter corners. $C_\text E{=}4$ is the surface correction for energy. In the following, we calculate the spectral energy density in the frequency band 8 Hz to 16 Hz.


In [ ]:
def spectral_energy_density(stream, f1, f2, rho=2700, fs=4):
    """
    Return trace with spectral energy density in specified frequency band
    
    :param stream: stream of a 3 component seismogram
    :param f1, f2: lower, upper corner frequency of bandpass
    :param rho: density (kg/m**3)
    :param fs: free surface correction (default: 4)
    :return: trace with total energy density, units: J/m**3/Hz
    """
    assert len(stream) == 3
    stream = stream.copy().detrend('linear').filter('bandpass', freqmin=f1, freqmax=f2, zerophase=True)    
    data = [envelope(tr).data ** 2 for tr in stream]
    data = np.sum(data, axis=0)
    data = rho * data / 2 / (f2 - f1) / fs
    tr = stream[0]
    tr.data = data
    tr.stats.channel = tr.stats.channel[:2] + 'X'
    return tr

tr = spectral_energy_density(stream, 8, 16)
plt.semilogy(t, tr.data)
plt.xlabel('time after rupture (s)')
ylabel = r'spectral energy density (J m$^{-3}$ Hz$^{-1}$)'
plt.ylabel(ylabel);


### Green's function

Next, we plot the spectral energy density and the radiative transfer Green's function together in a single plot. Using the analytical approximate solution for 3-d radiative transfer `G`, the parameters r (distance), t (time series), c (mean shear wave velocity) and g0 (transport scattering coefficient = inverse of transport mean free path l*) are expected (see help). The distance is calculated and reasonable parameters are set.

In [ ]:
from obspy.geodetics import gps2dist_azimuth
from qopen.rt import G

def calc_distance(ev, stacode):
    """Distance between event and station"""
    # find network code of station
    netcode = [net.code for net in inv for sta in net if sta.code == stacode][0]    
    coords = inv.get_coordinates(f'{netcode}.{stacode}..HHZ')
    hdist, _, _ = gps2dist_azimuth(coords['latitude'], coords['longitude'],
                                   ev['lat'], ev['lon'])
    dist = (hdist ** 2 + (1000 * ev['dep'] - coords['local_depth'])**2) ** 0.5
    return dist


help(G)

dist = calc_distance(ev, sta)
print(f'Distance between event {evid} and station {sta} is {dist/1000:.2f}km.')
vs = 3200  # 3.5 km/s shear wave velocity
l = 60e3  # 60 km transport mean free path


plt.semilogy(t, tr.data, label='observed')
plt.plot(t, G(dist, t, vs, 1 / l), label=f"Green's function for l*={l/1000:.0f}km")
plt.xlabel('time after rupture (s)')
plt.ylabel(ylabel)

plt.legend();

### Impact of scattering and intrinsic attenuation on the modeled envelope 

The Green's function G has units of m$^{-3}$. In the following, we simply scale it with the coda normalization procedure to go to the same units as the observed energy density.
We also add intrinsic attenuation to the equation. The effect on the energy envelope is given by $e^{-t/t_\text a}$ with absorption time $t_\text a$.

The coda normalization factor accounts for the site gain and source terms. It is approximately equal to the spectral source energy W(f) if the attenuation parameters are not too far away from reality. Contrarily, in Qopen the full S wave envelope is used to determine W.

*Please edit the following cell to include the coda normalization. Then add the intrinsic attenuation term and try to get a similar shape for the observed and modeled energy by adjusting the intrinsic absorption value.*

In [ ]:
def plot_energy(l0, ta=1e10, codanorm_time=None):
    """
    Plot observed energy density and scaled Green's function

    :param l0: transport mean free path (m)
    :param ta: absorption time (s), default: no absorption
    :codanorm_time: 2-dim tuple (s) specifying time window for
        coda normalization of the Green's function
    """
    # TODO: add intrinsic attenuation term, ta is absorption time
    plt.semilogy(t, tr.data, label='observed')
    Emod = G(dist, t, vs, 1 / l0)
    if codanorm_time is not None:
        codanorm_ind = np.logical_and(
            t >= codanorm_time[0], t < codanorm_time[1])
        codanorm = np.mean(tr.data[codanorm_ind]) / np.mean(Emod[codanorm_ind])
        print(f'Coda normalization factor is {codanorm:.1e} J/Hz  '
              '(approx. spectral source energy W(f))')
        Emod = Emod * codanorm
    plt.plot(t, Emod, label=f"Green's function for l*={l0/1000:.0f}km")
    plt.xlabel('time after rupture (s)')
    plt.ylabel(ylabel)
    plt.xlim(-20, 80)
    if codanorm_time is not None:
        plt.ylim(np.min(tr.data), None)
    plt.legend()


plot_energy(60e3)

It is impossible to obtain a good fit between model and observation near the onset of the direct S-wave. In the present scattering regime, waves arriving after the onset are forward scattered waves, which cannot be modeled with the isotropic scattering model underlying the Green's function used. Instead, it is sufficient to compare the mean values in a time window around the S-onset (see e.g. Gaebler et al., 2015).

## 3) Hands-on Qopen

Now we have a basic idea of how the scattering and absorption parameters (l*, ta) and the additional parameters R (site amplification for each station) and W (spectral source energy) are estimated by minimizing the system of equations
$$E_\text{obs} = R(r) W G(r, t, l^*) \exp(-t/t_\text a) \tag 3$$
for each frequency band and event.


Please open a text editor and look at the Qopen configuration file `conf.json` in the `qopen` folder. The most important options define the input files (events, stations, waveforms) and the time windows used (S wave time window, coda time window).

After that you can test Qopen with a single event by running the following command inside the `qopen` folder (for Windows adapt path separator):

```
qopen go -e 38445975 --prefix "00_test/"
```

We add the `--prefix` option to write all output to a separate directory (see `qopen go -h`).
If that worked we can run Qopen on the selected dataset of 6 stations and 8 events with

```
qopen go --prefix "01_go/" -vv
```

Now it is time to check the results and plots in the `01_go` directory.

In [ ]:
!ls -R qopen/01_go


Results are written to the `results.json` file, logs to `log.txt`.

Please look at all generated plots and understand what they show.


### Loading Qopen results

The results file in JSON format can be loaded using Python's json module. The averaged attenuation parameters are loaded and visualized as transport mean free path and absorption length as a function of frequency.

In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
from os.path import normpath

vs = 3200

with open(normpath('qopen/01_go/results.json')) as f:
    results = json.load(f)

print('Contents of JSON result file', list(results.keys()))

f = np.array(results['freq'])
g0 = np.array(results['g0'])
b = np.array(results['b'])

plt.figure(figsize=(6,3))
plt.plot(f, 1/g0/1000, marker='.', label='transport mean free path')
plt.plot(f, 1/b*vs/1000, marker='.', label='absorption length')
plt.xlabel('frequency (Hz)')
plt.ylabel('length (km)');
plt.legend();

The relationship between $Q$ and the attenuation parameters is given by (left scattering strength, right intrinsic attenuation)
$$\begin{aligned}Q_\text{sc}^{-1} &= \frac{g^* v_\text s}{2\pi f} & Q_\text i^{-1} &= \frac{b}{2\pi f}\end{aligned} \tag 4$$
In the next cell, we also compute and plot robust error estimates. To do this, the b and g* estimates for each event must be retrieved from the results dictionary. The inverse of Q is plotted against frequency in a log-log plot. Note that we have used only a subset of the available data. The attenuation parameters can be estimated with a larger data set by editing the configuration file or by adding the `--events` and `--inventory` flags and re-running `qopen go` (ideally with meaningful `--prefix` option).

In [ ]:
from math import pi
from qopen.util import gerr

def collect_eventresults(results):
    gs = []
    bs = []    
    for evid, res in results['events'].items():
        if res is None:
            continue
        gs.append(res['g0'])
        bs.append(res['b'])
    return np.array(gs), np.array(bs)

def plotQ(Qc=None):
    plt.figure(figsize=(6,3))
    ax = plt.subplot(111)
    ax.errorbar(f, Qscm, Qscerr1, Qscerr2, marker='.',  label='scattering')
    ax.errorbar(f, Qim, Qierr1, Qierr2, marker='.', label='intrinsic')
    ax.plot(f, Qscm+Qim, color='0.5', marker='.', label='total')
    if Qc:
        ax.plot(f, Qc, marker='.', label='coda')
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_xlabel('frequency (Hz)')
    ax.set_ylabel(r'$Q^{-1}$')
    ax.legend()
    ax.set_xticks((3, 10, 50))
    ax.set_xticklabels(('3', '10', '50'));    

gs, bs = collect_eventresults(results)
Qsc = gs * vs / (2 * pi * f)  # inverse of Qsc
Qi = bs / (2 * pi * f)  # inverse of Qi
# calculate the robust mean and error
Qscm, Qscerr1, Qscerr2 = gerr(Qsc, axis=0, robust=True)
Qim, Qierr1, Qierr2 = gerr(Qi, axis=0, robust=True)
plotQ()

Playground for inspecting the `results` object saved in the `results.json` file:

In [ ]:
results

### Comparison with coda Q

The interpretation of coda Q depends on the scattering regime. Let us calculate coda Q for a single event at a single station and compare it with the Q values plotted above.

The energy envelope is corrected for geometrical spreading with the factor $t^{1.5}$. The slope bc of a simple linear fit to the logarithm of the envelope is the "coda attenuation" factor, which can be converted to Qc with formula (4, right).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from obspy import read, UTCDateTime as UTC
from os.path import normpath
from qopen.util import linear_fit
# We reuse the function spectral_energy_density()
# but the scaling does not matter
arbitrary_energy = spectral_energy_density

def codaQ(t, env, f, tw=(20, 40)):
    """
    Calculate and return inverse of coda Q by fitting a log energy envelope
    
    :param t: array with lag times
    :param env: Trace with energy envelope (a.u.)
    :param tw: tuple of seconds specifying the time window
    """
    ind = np.logical_and(t >= tw[0], t < tw[1])
    t = t[ind]
    y = np.log(env.data[ind] * t ** 1.5)
    bc, offset = linear_fit(y, t)
    Qc = -bc / 2 / np.pi  / f
    return Qc, -bc, offset, t

evid = '38445975'
sta = 'CLC'
origin_time = UTC('2019-07-05T00:18:01.000')
stream = read(normpath(f'data/waveforms/{evid}/*{sta}*.ms'))
t = stream[0].times(reftime=origin_time)

fc = 12
env = arbitrary_energy(stream, 2/3*fc, 4/3*fc)
Qc, bc, offset, tind = codaQ(t, env, fc)

plt.semilogy(t, env.data, label='envelope')
env_geom = env.copy()
env_geom.data = env_geom.data[t>1] * t[t>1] ** 1.5
plt.semilogy(t[t>1], env_geom.data, label=r'envelope corrected with t$^{1.5}$')
plt.semilogy(tind, np.exp(-bc*tind+offset), 'k', label=rf'fit bc={bc:.3f}s$^{{-1}}$')
plt.xlabel('time after rupture (s)')
plt.ylabel('energy (a.u.)')
plt.legend();
plt.xlim(-10, 100);

This procedure can be repeated for all frequency for that we estimated the attenuation parameters and the results are inserted into the figure displaying inverse Q values.
Note that coda Q was determined here only for a single event-station pair.

*Why does Q$_c^{-1}$ decrease drastically for high frequencies? Fix the issue.*

In [ ]:
Qcs = []
tws = [(20, 40), (20, 40), (20, 40), (20, 40), (20, 40)]
for fc, tw in zip(f, tws):
    env = arbitrary_energy(stream, 2/3*fc, 4/3*fc)
    Qc = codaQ(t, env, fc, tw=tw)[0]
    Qcs.append(Qc)
plotQ(Qc=Qcs);

### Site amplification and earthquake source properties

While playing with the Qopen results we noticed, that both the site amplification (key `"R"`) and the source displacement spectra (key `"sds"`) are calculated by Qopen. Since b and g* are parameters of the medium, it makes sense to fix these two parameters if we want to put the focus on the simple gain factors (site and source). This can be done using Qopen's `fixed` command.
Note, that the shape of the envelope is now fully constrained by the attenuation parameters. There is still an ambiguity between site and source. The perfect tradeoff between the site and source terms in equation (3) means that a site term scaled by some factor at all stations leads to an equivalent scaling of the spectral source energy by the inverse of that factor, and together they explain the observations just as well. To solve this problem, we define a reference site and set the site amplification at that site to 1. By default, the geometric mean of the site gain at all stations is set to 1, but it may be more appropriate to use a borehole station or a station with known low site gain as the reference site.

After this step, we can take this approach even further -- site amplifications should also be fixed -- and we can recalculate the source spectra under these circumstances by fitting a single parameter $W(f)$ for each event and frequency band. In this way, source spectra of new earthquakes can also be estimated in a fast and robust manner. The flowchart below demonstrates the different steps and the corresponding invocation of Qopen commands.

![Qopen flowchart](qopen_flow.svg)

When using the `fixed` command, we need to specify the attenuation parameters. This is done with the `--input` option pointing to the results file of the previous run. Also, we do not want to plot attenuation results, and we select the station MPM as the reference site. This is an arbitrary choice. We could make a better and more informed choice, if we processed all 34 available stations. *Please execute the following command in the `qopen` folder:*

```
qopen fixed --input "01_go/results.json" --prefix "02_sites/" --no-plot-results --align-sites --align-sites-station "CI.MPM" -vv
```

Please compare the `sites.pdf` and `sds.pdf` and `mags.pdf` plots of the two different Qopen runs. For the last step, we will run the `source` command once with the 8 earthquakes, and we will run the same command for the 56 events in a retrospective monitoring mode. For this we use the `--events` parameter. In general, CLI parameters take precedence over configuration options. Please refer to the CLI help (e.g. `qopen source -h`) and the [API documentation](https://qopen.readthedocs.io), as all parameters are passed to the corresponding functions, for which you can find documentation there. We also specify the site amplification results from the previous run with the `--input-sites` parameter.

```
qopen source --input "01_go/results.json" --input-sites "02_sites/results.json" --prefix "03_source/" --seismic-moment-options '{"fc": null, "n": 2.58, "gamma": 2}'  --no-plot-results -vv

qopen source --events "../data/events56.csz" --input "01_go/results.json" --input-sites "02_sites/results.json" --prefix "04_source_all/" --seismic-moment-options '{"fc": null, "n": 2.58, "gamma": 2}'  --no-plot-results -vv

```

When fitting the source model to the source displacement spectrum, the high-frequency fall-off $n$ was fixed, as there is a large trade-off between $n$ and corner frequency. Spectra of the 56 events can be viewed in the new `sds.pdf` file.

In the following cell, we load the events from the results file and plot a simple scaling between local magnitude and moment magnitude estimated by Qopen.

In [ ]:
import json
from os.path import normpath
import matplotlib.pyplot as plt
import numpy as np
from qopen.imaging import _secondary_yaxis_seismic_moment
from qopen.util import linear_fit


with open(normpath('qopen/04_source_all/results.json')) as f:
    results = json.load(f)

params = [(evid, ev['Mw'], ev['Mcat'], ev['fc'])
          for evid, ev in results['events'].items()]
dtype=[('id', '<U20'), ('Mw', float), ('Ml', float), ('fc', float)]
params = np.array(params, dtype=dtype)

plt.plot(params['Ml'], params['Mw'], 'x')
a, b = linear_fit(params['Mw'], params['Ml'])
m = np.linspace(*plt.xlim(), 100)
Mw = r'M$_\mathrm{w}$'
Ml = r'M$_\mathrm{L}$'
plt.plot(m, a * m + b, label=f'{Mw} = %.2f{Ml} %+.2f' % (a, b))

_secondary_yaxis_seismic_moment(plt.gca())
plt.xlabel(f'local magnitude {Ml}')
plt.ylabel(f'moment magnitude {Mw}')
plt.legend();

## 4) Bonus: More stations, more frequency bands

In general, the processing can be improved in two ways: (1) using all stations of the dataset and, more importantly, (2) using more frequencies from 0.5 Hz up to the Nyquist frequency. When analyzing source parameters, the next step would be to check the statistics of source parameters (seismic moment, corner frequency, stress drop).

Please update the configuration file and reprocess the dataset using 13 central frequencies of `0.75, 1.06, 1.5, 2.1, 3.0, 4.2, 6.0, 8.5, 12.0, 17.0, 24.0, 33.9, 48.0` and all stations. Align sites relative to three stations MPM, WBM, WMF with the option `--align-sites-station "CI.MPM,CI.WBM,CI.WMF"` in the `fixed` command.
You can use the code in this notebook to analyze updated results.

```

```

## References

See https://github.com/trichter/qopen#references

The Qopen Ridgecrest repository can be found at https://github.com/trichter/qopen_ridgecrest